# **MPA-MLF - Miniproject**

## **Task - Classification of wireless transmitters**

The task will be to classify wireless transmitters based on data from measurement.

The dataset consists of 19200 samples and 9 features in total.

## Get Data

In [ ]:
!wget https://github.com/xlukac14/MPA-MLF/blob/main/Miniproject/Data_prepared_final.csv

#!wget --no-check-certificate \
#   https://github.com/xlukac14/MPA-MLF/blob/main/Miniproject/Data_prepared_final.csv

## Import Libraries

Import all libraries used for this miniproject. (some libraries might stay here unused:)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LeakyReLU, Dropout
from keras.optimizers import SGD, Adam, Nadam, Adamax
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn import preprocessing

import pandas as pd
import numpy as np
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib

## Load the Dataset

Loading the dataset from the path

In [ ]:
path_to_dataset = '/content/Data_prepared_final.csv' # change the PATH
pd_dataset = pd.read_csv(path_to_dataset)

## Data Examination

Reading data from variable dataset

In [ ]:
pd_dataset

,Unnamed: 0,time [s],cfo_meas,cfo_demod,gain_imb,iq_imb,or_off,quadr_err,ph_err,mag_err,evm
0,0,45181.524,-414,-421.076324,0.076097,-46.987598,-27.219631,0.104362,1.129778,0.440299,2.059935
1,1,45182.129,-414,-423.571686,0.069879,-47.852627,-26.934967,0.053158,1.050010,0.431240,1.815053
2,2,45182.550,-414,-421.033936,0.061466,-48.968697,-27.130722,-0.045913,0.989473,0.447790,1.762053
3,3,45182.971,-414,-420.103546,0.078836,-46.756973,-27.005318,0.081583,1.085091,0.443476,1.894204
4,4,45183.551,-432,-420.781097,0.066053,-47.048218,-27.358521,0.081920,1.023701,0.465225,1.805841
...,...,...,...,...,...,...,...,...,...,...,...
19195,19195,54245.724,216,215.584991,0.077551,-35.409054,-28.611925,1.875334,0.999375,1.427339,2.248822
19196,19196,54246.340,216,215.247406,0.044961,-35.508205,-27.954048,1.898758,1.031501,1.478366,2.320668
19197,19197,54246.770,216,216.761307,0.070066,-35.038486,-28.467575,2.015116,1.090227,1.468599,2.382019
19198,19198,54247.379,216,215.353516,0.048611,-35.413639,-28.528139,1.916172,0.999381,1.497681,2.283976


In [ ]:
pd_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19200 entries, 0 to 19199
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  19200 non-null  int64  
 1   time [s]    19200 non-null  float64
 2   cfo_meas    19200 non-null  int64  
 3   cfo_demod   19200 non-null  float64
 4   gain_imb    19200 non-null  float64
 5   iq_imb      19200 non-null  float64
 6   or_off      19200 non-null  float64
 7   quadr_err   19200 non-null  float64
 8   ph_err      19200 non-null  float64
 9   mag_err     19200 non-null  float64
 10  evm         19200 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 1.6 MB


### Columns dropping (Unnamed: 0 and time [s]).

In [ ]:
drop_columns = ['Unnamed: 0', 'time [s]']
pd_dataset = pd_dataset.drop(drop_columns, axis=1)

In [ ]:
print('pd_dataset:', pd_dataset.shape)

pd_dataset: (19200, 9)


In [ ]:
pd_dataset.head(11)   # dispalys 1st 11 rows

,cfo_meas,cfo_demod,gain_imb,iq_imb,or_off,quadr_err,ph_err,mag_err,evm
0,-414,-421.076324,0.076097,-46.987598,-27.219631,0.104362,1.129778,0.440299,2.059935
1,-414,-423.571686,0.069879,-47.852627,-26.934967,0.053158,1.050010,0.431240,1.815053
2,-414,-421.033936,0.061466,-48.968697,-27.130722,-0.045913,0.989473,0.447790,1.762053
3,-414,-420.103546,0.078836,-46.756973,-27.005318,0.081583,1.085091,0.443476,1.894204
4,-432,-420.781097,0.066053,-47.048218,-27.358521,0.081920,1.023701,0.465225,1.805841
5,-432,-416.307983,0.093724,-45.336712,-27.172468,0.045200,1.067799,0.480190,1.903123
6,-414,-422.121307,0.074381,-47.157940,-27.254124,-0.109112,1.065073,0.473195,1.862016
7,-414,-423.967285,0.094833,-45.233143,-26.917641,-0.047091,1.039195,0.465907,1.810157
8,-432,-424.394470,0.085833,-45.142429,-26.951569,-0.055014,1.008745,0.427296,1.751380
9,-432,-420.001892,0.062082,-49.143627,-27.330563,0.062528,1.041857,0.454947,1.878264


In [ ]:
#reshaped_data = pd_dataset.reshape((9, 1))

## Data Preprocessing

In [ ]:
scale = preprocessing.MinMaxScaler()

pd_dataset_column = pd_dataset.columns
pd_dataset_values = scale.fit_transform(pd_dataset.values)
pd_dataset = pd.DataFrame(pd_dataset_values, columns = pd_dataset_column)

In [ ]:
pd_dataset.head(11)

,cfo_meas,cfo_demod,gain_imb,iq_imb,or_off,quadr_err,ph_err,mag_err,evm
0,0.452174,0.448203,0.798527,0.332758,0.936888,0.578083,0.025894,0.005213,0.023679
1,0.452174,0.447015,0.786037,0.301680,0.956561,0.571407,0.018358,0.004325,0.009692
2,0.452174,0.448223,0.769136,0.261584,0.943033,0.558490,0.012638,0.005946,0.006664
3,0.452174,0.448666,0.804030,0.341043,0.951699,0.575113,0.021672,0.005524,0.014213
4,0.443478,0.448344,0.778349,0.330580,0.927289,0.575157,0.015872,0.007654,0.009166
5,0.443478,0.450474,0.833939,0.392068,0.940148,0.570369,0.020038,0.009120,0.014722
6,0.452174,0.447705,0.795081,0.326638,0.934504,0.550249,0.019781,0.008435,0.012374
7,0.452174,0.446826,0.836167,0.395789,0.957759,0.558336,0.017336,0.007721,0.009412
8,0.443478,0.446623,0.818087,0.399048,0.955414,0.557303,0.014459,0.003939,0.006055
9,0.443478,0.448715,0.770373,0.255299,0.929222,0.572629,0.017587,0.006647,0.013302


## Model building and its architecture

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(9, 1)))   # input dimension
#model.add(Dropout(0.2))
#model.add(Dense(256, activation='selu', kernel_regularizer=regularizers.L1L2(l1=1e-6, l2=1e-5)))   # selu has been best so far
#model.add(Dense(256, activation='selu', kernel_regularizer=regularizers.L1L2(l1=1e-6, l2=1e-5)))   # selu has been best so far
#model.add(Dense(10, activation='softmax'))
#model.add(Flatten(input_shape=(32, 32, 3)))
#model.add(Dense(9, activation='gelu'))
model.add(Dense(64, activation='selu'))
model.add(Dense(9, activation='softmax'))

#model = Sequential([
#    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(3, 3, 1)),
    # Add more layers as needed
#])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 9)                 0         
                                                                 
 flatten_7 (Flatten)         (None, 9)                 0         
                                                                 
 dense_6 (Dense)             (None, 9)                 90        
                                                                 
 dense_7 (Dense)             (None, 9)                 90        
                                                                 
Total params: 180 (720.00 Byte)
Trainable params: 180 (720.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



### Optimization

In [ ]:
optimizer = Adamax(learning_rate = 0.01, epsilon = 0.2, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Model Training

In [ ]:
history = model.fit(pd_dataset, epochs=40, batch_size=50, validation_split = 0.2)

Epoch 1/40


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1152, in train_step
        self._validate_target_and_loss(y, loss)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1106, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=categorical_crossentropy, and therefore expects target data to be provided in `fit()`.


## Model Evaluation

In [ ]:
score = model.evaluate(pd_dataset, verbose=0)
print('Test loss:', score[0])
print(f'Test accuracy: {score[1]*100} %')

### Figures

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

## Output

In [ ]:
prediction = model.predict(pd_dataset)

# Save the predictions to a CSV file
prediction = prediction.argmax(axis=1).astype(np.uint8)+1
output_df = pd.DataFrame(prediction, columns=["cfo_meas"])

output_path = 'https://github.com/xlukac14/MPA-MLF/tree/main/Miniproject/results.csv'
output_df.to_csv(output_path, index=True, index_label="id", quoting=2)